https://milumon.medium.com/workshop-quickstart-to-speech-recognition-google-speech-to-text-with-colab-for-beginners-0996bc8279e6

In [ ]:
!pip install google-cloud
!pip install google-cloud-speech
!pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.4/289.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
from google.colab import auth
from google.cloud import speech
import pandas as pd
import soundfile as sf
import wave
import re

In [1]:
import os
import pandas as pd

In [3]:
os.chdir('/content/drive/MyDrive/BI Hackathon/Real Testing')

In [ ]:
auth.authenticate_user(project_id='turing-team')

In [4]:
df = pd.read_csv('output.csv')

In [5]:
df

,sentence,label,filename
0,harga seratus lima puluh ribu,Rp 150.000,audio1.wav
1,beli sepuluh ribu,Rp 10.000,audio2.wav
2,total seratus ribu,Rp 100.000,audio3.wav
3,bayar dua juta,Rp 2.000.000,audio4.wav
4,harga barang dua ratus ribu,Rp 200.000,audio5.wav
...,...,...,...
895,harga barang sepuluh ribu,Rp 10.000,audio896.wav
896,beli dua ratus ribu,Rp 200.000,audio897.wav
897,total seratus lima puluh ribu,Rp 150.000,audio898.wav
898,diskon seratus ribu,Rp 100.000,audio899.wav


In [ ]:
client = speech.SpeechClient()

def strip_number(text):
    stripped_text = re.sub(r'[^\d,\.]', '', text)

    if stripped_text:
        parts = stripped_text.split(',')
        if len(parts) > 1:
            main_part = parts[0].replace('.', '')
            decimal_part = parts[1]
            formatted_number = f"{int(main_part):,}.{decimal_part}".replace(',', '.')
        else:
            formatted_number = f"{int(stripped_text.replace('.', '')):,}".replace(',', '.')
        return f"Rp {formatted_number}"
    return stripped_text

def process_audio(file_name):
    try:
        data, sample_rate = sf.read(file_name)
        audio_content = (data * (2**15)).astype("<h").tobytes()
        audio = speech.RecognitionAudio(content=audio_content)

        config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=sample_rate,
            language_code="id-ID",
            model="default",
            audio_channel_count=1,
            enable_word_time_offsets=True,
        )

        operation = client.long_running_recognize(config=config, audio=audio)
        response = operation.result(timeout=90)

        for result in response.results:
            transcript = result.alternatives[0].transcript
            return transcript

    except Exception as e:
        print(f"Failed to process {file_name}: {e}")
        return None

In [ ]:
predicted_full = []
predicted = []
prediction_result = []

for index, row in df.iterrows():
    file_name = os.path.join("generated_audio", row['filename'])
    transcript = process_audio(file_name)

    if transcript is not None:
        predicted_full.append(transcript)
        stripped_transcript = strip_number(transcript)
        predicted.append(stripped_transcript)
        prediction_result.append(1 if stripped_transcript == row['label'] else 0)
    else:
        predicted_full.append(None)
        predicted.append(None)
        prediction_result.append(None)

    intermediate_df = df.iloc[:len(predicted_full)].copy()
    intermediate_df['predicted_full'] = predicted_full
    intermediate_df['predicted'] = predicted
    intermediate_df['prediction_result'] = prediction_result

    intermediate_df.to_csv('intermediate_results.csv', index=False)